In [1]:
# pip install transformers==4.47.1 # QUESTA VERSIONE IN QUESTO NOTEBOOK EVITA L'ERRORE: NameError: name 'MistralConfig' is not defined

In [2]:
import transformers
print(transformers.__version__)

/raid1_8TB/jupyter_shared/users/vincenzo-deleo/venv/venv_jupyterlab-python-3.12.8/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.47.1


In [3]:
experiment_name = '2025.01.18-02_Mistral_Nemo_12B_zero_shot'

In [4]:
pip install -q datasets --progress-bar off

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import pickle
from datasets import Dataset

In [6]:
# from google.colab import drive
# from IPython.display import Image, display

# mount_point = "/content/gdrive"
# base_path = mount_point + "/MyDrive/Colab/SDG/data"

# drive.mount(mount_point, force_remount=True)

In [7]:
base_path = "/raid1_8TB/jupyter_shared/projects/SDG/data"

In [8]:
output_data_path = base_path + "/../AnalisiTesti/FineTuning/Mistral_Nemo_12B/data/output/" + experiment_name

In [9]:
df = pd.read_excel(base_path + "/../AnalisiTesti/data/output/RisultatiSentiment_5modelli_CON_LABEL_FINALE.xlsx")
df

,Unnamed: 0.1,Unnamed: 0,text_id,text,sdg_true,sentiment_Llama3,sentiment_Mixtral,sentiment_Phi3,sentiment_Gemma,sentiment_Qwen,vote_level,label
0,0,0,1528,"Indeed, available empirical evidence typically...",10,neutral,neutral,negative,negative,negative,Silver,negative
1,1,1,2389,Having a secure and up-to-date knowledge datab...,15,neutral,positive,positive,positive,positive,Gold,positive
2,2,2,5801,It may be commonplace to state that the world ...,16,neutral,neutral,neutral,positive,negative,Silver,neutral
3,3,3,420,From the environmental protection point of vie...,12,positive,positive,positive,positive,positive,Platinum,positive
4,4,4,5138,"In other countries, such as the Czech Republic...",10,neutral,neutral,positive,positive,positive,Silver,positive
...,...,...,...,...,...,...,...,...,...,...,...,...
6395,6395,6395,2870,1. What's morality got to do with it? making t...,16,neutral,neutral,neutral,neutral,neutral,Platinum,neutral
6396,6396,6396,6330,As an organising principle within multi-level ...,16,neutral,neutral,neutral,positive,neutral,Gold,neutral
6397,6397,6397,2723,This article examines the neo-conservative cri...,16,neutral,neutral,neutral,negative,negative,Silver,neutral
6398,6398,6398,6710,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,neutral,neutral,neutral,positive,neutral,Gold,neutral


In [10]:
# BISOGNA RIMUOVERE DAL DATASET I TESTI UTILIZZATI PER L'ETICHETTATURA MANUALE DEL DATASET DI TEST
df_test = pd.read_excel(base_path + "/../AnalisiTesti/data/output/RisultatiSentiment_EtichetatturaManuale_DatasetDiTest_CON_LABEL_FINALE.xlsx").drop(columns=['Unnamed: 0'])
df_test

,text_id,text,sdg_true,label
0,210,The Brazilian Instituto de Pesquisa Economica ...,1,positive
1,251,While the growth effect predominated in practi...,1,neutral
2,441,"Nevertheless, the use of indices as a form of ...",1,positive
3,591,"If it is high, then public policies designed t...",1,positive
4,1250,Results for countries using national panels ad...,1,neutral
...,...,...,...,...
571,5801,It may be commonplace to state that the world ...,16,neutral
572,6119,Sexual citizenship and sexual rights scholarsh...,16,positive
573,6551,Legal pluralism is a phenomenon defined differ...,16,positive
574,6618,"This article discusses the extent, impacts, an...",16,negative


In [11]:
# BISOGNA RIMUOVERE DAL DATASET I TESTI UTILIZZATI PER L'ETICHETTATURA MANUALE DEL DATASET DI TEST
df = df[~df.text_id.isin(df_test.text_id)]
df

,Unnamed: 0.1,Unnamed: 0,text_id,text,sdg_true,sentiment_Llama3,sentiment_Mixtral,sentiment_Phi3,sentiment_Gemma,sentiment_Qwen,vote_level,label
0,0,0,1528,"Indeed, available empirical evidence typically...",10,neutral,neutral,negative,negative,negative,Silver,negative
1,1,1,2389,Having a secure and up-to-date knowledge datab...,15,neutral,positive,positive,positive,positive,Gold,positive
3,3,3,420,From the environmental protection point of vie...,12,positive,positive,positive,positive,positive,Platinum,positive
4,4,4,5138,"In other countries, such as the Czech Republic...",10,neutral,neutral,positive,positive,positive,Silver,positive
5,5,5,3504,They also show causality in both directions (t...,7,positive,positive,positive,positive,positive,Platinum,positive
...,...,...,...,...,...,...,...,...,...,...,...,...
6394,6394,6394,2370,Purpose – This article aims to explore the con...,16,positive,neutral,positive,positive,positive,Gold,positive
6396,6396,6396,6330,As an organising principle within multi-level ...,16,neutral,neutral,neutral,positive,neutral,Gold,neutral
6397,6397,6397,2723,This article examines the neo-conservative cri...,16,neutral,neutral,neutral,negative,negative,Silver,neutral
6398,6398,6398,6710,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,neutral,neutral,neutral,positive,neutral,Gold,neutral


To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Mistral Nemo 12b is the largest OSS model to fit on a Colab Tesla T4!**

In [12]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [13]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Mistral-Nemo-Instruct-2407",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Mistral patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090 Ti. Max memory: 23.679 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.07s/it]


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [14]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.5 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` or `mistral` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [15]:
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatting_prompts_func(examples):
#     instructions = examples["instruction"]
#     inputs       = examples["input"]
#     outputs      = examples["output"]
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }
# pass

# from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

In [16]:
# Creazione del dataset di train

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Prompt generale per la classificazione
prompt_instr = """Given the following input text, between triple quotes, with its associated classification label with respect to one of the Sustainable Development Goals (SDGs), further classify the text with respect to the three labels defined below:

  - "positive": the text implies or explicitly states that what is affirmed or described leads to a significant advancement in favor of the goal indicated by the SDG under which it was previously classified.

  - "negative": the text implies or explicitly states that what is affirmed or described leads to a significant advancement against the goal indicated by the SDG under which it was previously classified.

  - "neutral": the text does not imply or explicitly state that what is affirmed or described leads to a significant advancement either in favor or against the goal indicated by the SDG under which it was previously classified.

Return the result in JSON format with 1 key:

  - "label": The assigned label for the input text [this key/value pair is mandatory]."""

# Preparazione dei dati per il dataset
data = {
    'output': [],
    'input': [],
    'instruction': []
}

# Itera attraverso il DataFrame per popolare il dataset
for _, row in df.iterrows():
    data['input'].append(f"""INPUT TEXT: \"{row["text"]}\"\nSDG_CLASSIFICATION: \"SDG-{row["sdg_true"]}\"""")
    data['output'].append(row["label"])
    data['instruction'].append(prompt_instr)

# Creazione del dataset utilizzando Hugging Face's Dataset
dataset = Dataset.from_dict(data)

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)



Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5824/5824 [00:00<00:00, 81500.95 examples/s]

Dataset({
    features: ['output', 'input', 'instruction', 'text'],
    num_rows: 5824
})


In [17]:
# Creazione del dataset di train

test_dataset_data_map_for_text_id_and_sdg_true = {}

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Prompt generale per la classificazione
prompt_instr = """Given the following input text, between triple quotes, with its associated classification label with respect to one of the Sustainable Development Goals (SDGs), further classify the text with respect to the three labels defined below:

  - "positive": the text implies or explicitly states that what is affirmed or described leads to a significant advancement in favor of the goal indicated by the SDG under which it was previously classified.

  - "negative": the text implies or explicitly states that what is affirmed or described leads to a significant advancement against the goal indicated by the SDG under which it was previously classified.

  - "neutral": the text does not imply or explicitly state that what is affirmed or described leads to a significant advancement either in favor or against the goal indicated by the SDG under which it was previously classified.

Return the result in JSON format with 1 key:

  - "label": The assigned label for the input text [this key/value pair is mandatory]."""

# Preparazione dei dati per il dataset
test_dataset_data = {
    'output': [],
    'input': [],
    'instruction': []
}

# Itera attraverso il DataFrame per popolare il dataset
for _, row in df_test.iterrows():
    test_dataset_data['input'].append(f"""INPUT TEXT: \"{row["text"]}\"\nSDG_CLASSIFICATION: \"SDG-{row["sdg_true"]}\"""")
    test_dataset_data['output'].append(row["label"])
    test_dataset_data['instruction'].append(prompt_instr)

    full_prompt = alpaca_prompt.format(prompt_instr,
                                       f"""INPUT TEXT: \"{row["text"]}\"\nSDG_CLASSIFICATION: \"SDG-{row["sdg_true"]}\"""",
                                       row["label"]) + EOS_TOKEN

    test_dataset_data_map_for_text_id_and_sdg_true[hash(full_prompt.strip().replace('\n', ' '))] = {"text_id": row["text_id"], "sdg_true": row["sdg_true"]}

# Creazione del dataset utilizzando Hugging Face's Dataset
test_dataset = Dataset.from_dict(test_dataset_data)

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

test_dataset = test_dataset.map(formatting_prompts_func, batched = True,)
print(test_dataset)



Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 576/576 [00:00<00:00, 60375.34 examples/s]

Dataset({
    features: ['output', 'input', 'instruction', 'text'],
    num_rows: 576
})


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [18]:
# from trl import SFTTrainer
# from transformers import TrainingArguments
# from unsloth import is_bfloat16_supported

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     dataset_num_proc = 2,
#     packing = False, # Can make training 5x faster for short sequences.
#     args = TrainingArguments(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         # max_steps = 3, # 60,
#         num_train_epochs = 1, # For longer training runs!
#         learning_rate = 2e-4,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
#         report_to = "none", # Use this for WandB etc
#     ),
# )

In [19]:
# #@title Show current memory stats
# gpu_stats = torch.cuda.get_device_properties(0)
# start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
# print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
# print(f"{start_gpu_memory} GB of memory reserved.")

In [20]:
# trainer_stats = trainer.train()

In [21]:
# #@title Show final memory and time stats
# used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
# used_percentage = round(used_memory         /max_memory*100, 3)
# lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
# print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
# print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
# print(f"Peak reserved memory = {used_memory} GB.")
# print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
# print(f"Peak reserved memory % of max memory = {used_percentage} %.")
# print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [22]:
# # alpaca_prompt = Copied from above
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "Continue the fibonnaci sequence.", # instruction
#         "1, 1, 2, 3, 5, 8", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, pad_token_id = tokenizer.eos_token_id)
# tokenizer.batch_decode(outputs)

In [23]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

cnt=0
for data in test_dataset:
  print("\ncnt: ", cnt)
  prompt = ""
  sdg_true = -1
  text_id = -1
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          data["instruction"], # instruction
          data["input"], # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  full_prompt = alpaca_prompt.format(data["instruction"],
                                       data["input"],
                                       data["output"]) + EOS_TOKEN

  # print("######################################################################################################################")
  # print(hash(full_prompt))
  # print("######################################################################################################################")
  # print(test_dataset_data_map_for_text_id_and_sdg_true.keys())
  # print("######################################################################################################################")

  sdg_true = test_dataset_data_map_for_text_id_and_sdg_true[hash(full_prompt.strip().replace('\n', ' '))]['sdg_true']
  text_id = test_dataset_data_map_for_text_id_and_sdg_true[hash(full_prompt.strip().replace('\n', ' '))]['text_id']

  # print(sdg_true)
  # print(text_id)

  outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True, pad_token_id = tokenizer.eos_token_id)

  # break


  generated_text = tokenizer.batch_decode(outputs)[0]

  result_extended = (cnt, text_id, sdg_true, generated_text)

  # print(generated_text)
  if 'json' in generated_text.lower():
    print(generated_text.lower().split('json')[-1])
  else:
    raise(Exception("JSON not found in generated text"))


  with open(output_data_path+f'/sdg_classification_{text_id}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)


  cnt+=1

  # break


cnt:  0
 format with 1 key:

  - "label": the assigned label for the input text [this key/value pair is mandatory].

### input:
input text: "the brazilian instituto de pesquisa economica aplicada (institute of applied economic research, ipea) has noted that every real (brl) invested in the programme increases gdp by brl 1.44. the 16 million children and adolescents whose school attendance is monitored by the programme show lower rates of truancy and are performing at a level equal to the average student in the public school system, despite their impoverished economic condition. this will lead to a future for these children far different from the situation of exclusion suffered by their parents and grandparents."
sdg_classification: "sdg-1"

### response:
{"label": "positive"}</s>

cnt:  1
 format with 1 key:

  - "label": the assigned label for the input text [this key/value pair is mandatory].

### input:
input text: "while the growth effect predominated in practically all tf appreci

In [24]:
STOP!

SyntaxError: invalid syntax (2635050600.py, line 1)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, max_new_tokens = 64, pad_token_id = tokenizer.eos_token_id)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>